In [32]:
import os
import cohere
import json
import random
import uuid
from IPython.display import Markdown, display
import time

In [ ]:
# generate embeddings for all files in a directory
cohere_api_key = os.getenv('COHERE_API_KEY')
co = cohere.ClientV2(cohere_api_key)

In [12]:
path = "./jsons"
# recursively get all files in the directory
files = []
for root, dirs, filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('.json'):
            relative_path = os.path.relpath(os.path.join(root, filename), path)
            files.append(relative_path)
print(len(files), 'files found')

valid_files = []
for f in files:
    # load json
    file_path = os.path.join(path, f)
    with open(file_path, 'r') as file:
        try:
            data = json.load(file)
        except json.JSONDecodeError:
            print(f"Error decoding JSON from file: {file_path}")
            continue
    # check if the json contains the error key
    # print(data.keys())
    if 'type' in data.keys():
        print(f"Key 'type' found in file: {file_path}")
        continue

    valid_files.append(f)

print(len(valid_files), 'valid files found')


382 files found
Key 'type' found in file: ./jsons/0-1095-1185-North-Talbot-Road/SDN-001_21 - Drawings.pdf.json
Key 'type' found in file: ./jsons/0-1095-1185-North-Talbot-Road/SDN-001_21 - Storm Water Management Report (updated).pdf.json
Key 'type' found in file: ./jsons/0-1095-1185-North-Talbot-Road/SDN-001_21 - Storm Water Management Report.pdf.json
Key 'type' found in file: ./jsons/0-1095-1185-North-Talbot-Road/SDN-001_21 - Phase 1 Environmental Site Assessment.pdf.json
Key 'type' found in file: ./jsons/1500-Ouellette-Avenue/Structural Integrity Report.pdf.json
Key 'type' found in file: ./jsons/0-Clairview-Avenue/Environmental Evaluation Report.pdf.json
Key 'type' found in file: ./jsons/0-Clairview-Avenue/Functional Servicing Report.pdf.json
Key 'type' found in file: ./jsons/0-Clairview-Avenue/Planning Justification Report.pdf.json
Key 'type' found in file: ./jsons/0-Clairview-Avenue/Species At Risk Considerations Memo.pdf.json
Key 'type' found in file: ./jsons/0-Clairview-Avenue/Sto

In [7]:
# select a random project
# project is subdirectories of path
def get_random_project():
    # get all subdirectories of path
    subdirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    # select a random subdirectory
    random_subdir = random.choice(subdirs)
    # get all files in the subdirectory
    return random_subdir

random_project = get_random_project()
print('random project:', random_project)
# get all files in this subdirectory
project_files = []
for root, dirs, filenames in os.walk(os.path.join(path, random_project)):
    for filename in filenames:
        if filename.endswith('.json'):
            relative_path = os.path.relpath(os.path.join(root, filename), path)
            project_files.append(relative_path)
print(len(project_files), 'files found in project', random_project)

random project: 3161-jefferson-boulevard-west-side-of-jefferson-south-of-queen-elizabeth-drive
11 files found in project 3161-jefferson-boulevard-west-side-of-jefferson-south-of-queen-elizabeth-drive


In [5]:
def create_chunks(data, chunk_size = 512, overlap=256):
    chunks = []
    for page in data["pages"]:
        text = page["markdown"]
        for i in range(0, len(text), chunk_size - overlap):
            chunk = text[i:i + chunk_size]
            chunks.append(chunk)
    return chunks

def create_smart_chunks(data, chunk_size=512, overlap=256):
    import re
    chunks = []
    page_numbers = {}
    # for page in data["pages"]:
    for page_number, page in enumerate(data["pages"]):
        text = page["markdown"]
        table_start_pattern = r"(\n\|)"
        table_end_pattern = r"(\|\n(?!\|))"
        
        # Find table starts and ends
        table_starts = [m.start() for m in re.finditer(table_start_pattern, text)]
        table_ends = [m.start() for m in re.finditer(table_end_pattern, text)]
        
        char_index = 0
        i = 0
        j = 0
        # for i in range(len(table_starts)):
        temp_chunks = []
        print(table_starts)
        while i < len(table_starts) and j < max(len(table_ends), 1):
            print("inside this loop")
            # Capture non-table text before the current table
            if table_starts[i] > char_index:
                print("capturing non-table text")
                non_table_chunk = text[char_index:table_starts[i]].strip()
                if non_table_chunk:
                    # call create_chunks on the non-table chunk
                    non_table_chunks = create_chunks({"pages": [{"index": page_number, "markdown": non_table_chunk}]}, chunk_size, overlap)
                    for chunk in non_table_chunks:
                        if chunk not in page_numbers:
                            page_numbers[chunk] = []
                        page_numbers[chunk].append(page_number+1)
                    temp_chunks.extend(non_table_chunks)
                    # char_index = table_starts[i]
                # continue
            
            # Capture the table chunk
            start = table_starts[i]
            end = table_ends[j] if j < len(table_ends) else len(text)

            char_index = end + 1
            j += 1
            i += 1
            # check if table_starts[i] is greater than table_ends[j]
            # if not skip table starts until it is
            while i < len(table_starts) and j < len(table_ends) and table_starts[i] < table_ends[j]:
                i += 1
            # add some pre-start context to the table:
            table_chunk = text[start:end+1].strip()
            if table_chunk:
                temp_chunks.append(table_chunk)
        
        # Capture any remaining non-table text after the last table
        if char_index < len(text):
            remaining_chunk = text[char_index:].strip()
            if remaining_chunk:
                # temp_chunks.append(remaining_chunk)
                # call create_chunks on the remaining chunk
                remaining_chunks = create_chunks({"pages": [{"index": page_number, "markdown": remaining_chunk}]}, chunk_size, overlap)
                for chunk in remaining_chunks:
                    if chunk not in page_numbers:
                        page_numbers[chunk] = []
                    page_numbers[chunk].append(page_number+1)
                temp_chunks.extend(remaining_chunks)
        # add page number to each chunk
        for chunk in temp_chunks:
            if chunk not in page_numbers:
                page_numbers[chunk] = []
                page_numbers[chunk].append(page_number+1)
        
        chunks.extend(temp_chunks)

    return chunks, page_numbers
# chunks, page_numbers = create_smart_chunks(mistral_response.json(), chunk_size=512, overlap=256)
# # chunks, page_numbers = create_smart_chunks(pdf_data, chunk_size=512, overlap=256)
# # if there are any chunks that are smaller than 512 characters, move them to the previous chunk
# for i in range(len(chunks)):
#     if len(chunks[i]) < 512:
#         # move the chunk to the previous chunk
#         if i > 0:
#             chunks[i-1] += "\n" + chunks[i]
#             chunks[i] = ""
# # remove empty chunks and page entries for removed chunks
# chunks = [chunk for chunk in chunks if chunk != ""]
# for chunk in chunks:
#     if chunk in page_numbers:
#         del page_numbers[chunk]
# create chunks for each file
def create_chunks_filepath(file_path):
    # read file as json
    with open(file_path, 'r') as file:
        data = json.load(file)
    # create chunks
    chunks, page_numbers = create_smart_chunks(data, chunk_size=512, overlap=256)
    # remove empty chunks
    chunks = [chunk for chunk in chunks if chunk != ""]
    # # if there are any chunks that are smaller than 512 characters, move them to the previous chunk
    smaller_chunks_available = True
    while(smaller_chunks_available):
        smaller_chunks_available = False
        for i in reversed(range(len(chunks))):
            if len(chunks[i]) < 512:
                # move the chunk to the previous chunk
                if i > 0:
                    if len(chunks[i])>0:
                        print([len(chunk) for chunk in chunks])
                        smaller_chunks_available = True
                        chunks[i-1] += "\n" + chunks[i]
                        chunks[i] = ""
            
    # remove empty chunks and page entries for removed chunks
    chunks = [chunk for chunk in chunks if chunk != ""]
    for chunk in chunks:
        if chunk in page_numbers:
            del page_numbers[chunk]
    # create chunks for each file
    return chunks, page_numbers
# create chunks for each file

In [8]:
chunks, page_number = create_chunks_filepath(os.path.join(path, project_files[0]))

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[25, 86, 50, 25, 25, 25, 25, 32, 25, 56]
[25, 86, 50, 25, 25, 25, 25, 32, 82, 0]
[25, 86, 50, 25, 25, 25, 25, 115, 0, 0]
[25, 86, 50, 25, 25, 25, 141, 0, 0, 0]
[25, 86, 50, 25, 25, 167, 0, 0, 0, 0]
[25, 86, 50, 25, 193, 0, 0, 0, 0, 0]
[25, 86, 50, 219, 0, 0, 0, 0, 0, 0]
[25, 86, 270, 0, 0, 0, 0, 0, 0, 0]
[25, 357, 0, 0, 0, 0, 0, 0, 0, 0]


In [59]:
print([len(chunk) for chunk in chunks])

[512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 954, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 958, 512, 512, 512, 798, 512, 831, 512, 512, 810, 536, 512, 512, 538, 512, 512, 512, 980, 512, 512, 834, 512, 512, 512, 512, 818, 512, 512, 512, 512, 512, 1023, 512, 512, 706, 512, 512, 512, 512, 858, 512, 512, 512, 512, 796, 512, 512, 512, 512, 512, 512, 512, 512, 628, 512, 512, 512, 512, 512, 512, 512, 512, 512, 702, 512, 512, 512, 512, 512, 526, 1017, 616]


In [60]:
# show chunks as markdown
def show_chunk(chunk):
    print("---------------")
    display(Markdown(chunk))
# show chunks as markdown
for chunk in chunks:
    show_chunk(chunk)

---------------


Office of the Commissioner of Economic Development \& Innovation Planning \& Development Services

# ZONING BY-LAW AMENDMENT APPLICATION 

## INSTRUCTIONS

## Verify that you are using the most current application form.

Section 1: Before this application can be submitted, you must complete both Stage 1 and Stage 2 Planning Consultation Applications.
Section 2: During the Stage 2 process, any required studies must be completed and submitted for review and comment. The final studies for Stage 2 must be inclu

---------------


can be submitted, you must complete both Stage 1 and Stage 2 Planning Consultation Applications.
Section 2: During the Stage 2 process, any required studies must be completed and submitted for review and comment. The final studies for Stage 2 must be included with this application and the proposal must remain unchanged.
Section 3: Provide the full name, address, phone number, fax number and email address of the applicant, agent, and registered owner. If any of these are a corporation, provide the full corpo

---------------


...

er against the City of Windsor, its officers, officials, employees or agents for or in respect of any loss, damage, injury or costs.

## Kushal Dighe

Name of Applicant (print)
![img-8.jpeg](img-8.jpeg)

Signature of Applicant

## February 27, 2024

Date

## Cindy Prince

Name of Agent (print)
![img-9.jpeg](img-9.jpeg)

Signature of Agent

## February 27, 2024

Date

## END OF SCHEDULE E
## Cindy Prince

Name of Agent (print)
![img-9.jpeg](img-9.jpeg)

Signature of Agent

## February 27, 2024

Date

## END OF SCHEDULE E


---------------


# ZONING BY-LAW AMENDMENT APPLICATION 

## DO NOT COMPLETE BELOW - STAFF USE ONLY

## Receipt and Assignment of Application

Date Received Stamp
This application has been assigned to:
Adam Szymczak (AS) $\square$ Brian Nagata (BN)
Frank Garardo (FG)
$\square$ Tracy Tang (TT)
$\square$ Jim Abbs (JA)
$\square$ Justina Nwaesei (JN)
$\square$ Kevin Alexander (KA)
$\square$ Laura Strahl (LS)
$\square$ Simona Simion (SS)
$\square$ $\qquad$

## Complete Application

This application is deemed complete on $\qquad$ 
re$ Tracy Tang (TT)
$\square$ Jim Abbs (JA)
$\square$ Justina Nwaesei (JN)
$\square$ Kevin Alexander (KA)
$\square$ Laura Strahl (LS)
$\square$ Simona Simion (SS)
$\square$ $\qquad$

## Complete Application

This application is deemed complete on $\qquad$ Date

Signature of Delegated Authority
$\square$ Neil Robertson, MCIP, RPP
Manager of Urban Design
$\square$ Greg Atkinson, MCIP, RPP
Manager of Development
$\square$ Thom Hunt, MCIP, RPP
City Planner \& Executive Director

## Internal Information


---------------


Date

Signature of Delegated Authority
$\square$ Neil Robertson, MCIP, RPP
Manager of Urban Design
$\square$ Greg Atkinson, MCIP, RPP
Manager of Development
$\square$ Thom Hunt, MCIP, RPP
City Planner \& Executive Director

## Internal Information
| Fee Paid: \$ | Receipt No: | Date: |
| :--: | :--: | :--: |
| Payment Type: $\square$ Cash | $\square$ Certified Cheque | $\square$ |
| NEW Zoning File No. ZNG/ | Z- |  |
| Previous Zoning File No. ZNG/ | Z- |  |
| Related OPA File No. OPA/ | OPA |  |
Other File Numbers: $\qquad$
Notes: $\qquad$
$\qquad$
$\qquad$

## THIS IS THE LAST PAGE OF THE APPLICATION FORM



In [14]:
# save chunks to file
chunks_path = "./chunks"
# cycle through all valid json files and create chunks, save chunks and page numbers to file
if not os.path.exists(chunks_path):
    os.makedirs(chunks_path)
# save chunks to file
for i, file in enumerate(valid_files):
    # check if sub-folder exists
    sub_folder = os.path.join(chunks_path, os.path.dirname(file))
    if not os.path.exists(sub_folder):
        os.makedirs(sub_folder)
    # if json file already exists, skip it
    if os.path.exists(os.path.join(chunks_path, file.replace('.json', '.chunks.json'))):
        print(f"File {i+1}/{len(project_files)}: {file} - already exists, skipping")
        continue
    
    # create chunks
    print(f"Creating chunks for file {i+1}/{len(project_files)}: {file}")
    chunks, page_numbers = create_chunks_filepath(os.path.join(path, file))
    # save chunks and page numbers to file
    with open(os.path.join(chunks_path, file.replace('.json', '.chunks.json')), 'w') as f:
        json.dump({"chunks": chunks, "page_numbers": page_numbers}, f)
    print(f"File {i+1}/{len(project_files)}: {file} - {len(chunks)} chunks created")


File 1/11: 8607-8649-8675-8699-McHugh-Street/Application Form Plan of Subdivision or Condominium.pdf.json - already exists, skipping
File 2/11: 8607-8649-8675-8699-McHugh-Street/Condominium Drawings.pdf.json - already exists, skipping
File 3/11: 8607-8649-8675-8699-McHugh-Street/DPC Exemption Cover Letter (December 2021).pdf.json - already exists, skipping
File 4/11: 0-1095-1185-North-Talbot-Road/SDN-001_21 - 1095 and 1185 North Talbot Survey.pdf.json - already exists, skipping
File 5/11: 0-1095-1185-North-Talbot-Road/SDN-001_21 - Acoustical Report.pdf.json - already exists, skipping
File 6/11: 0-1095-1185-North-Talbot-Road/SDN-001_21 - Bellocorp Development - North Talbot Tree Report.pdf.json - already exists, skipping
File 7/11: 0-1095-1185-North-Talbot-Road/SDN-001_21 - Draft Plan.pdf.json - already exists, skipping
File 8/11: 0-1095-1185-North-Talbot-Road/SDN-001_21 - Endangered Species Act Study.pdf.json - already exists, skipping
File 9/11: 0-1095-1185-North-Talbot-Road/SDN-001_2

In [15]:
print(chunks)

['# NOTICE OF PUBLIC MEETING <br> TO CONSIDER AMENDMENTS TO THE CITY OF WINDSOR OFFICIAL PLAN \n\n## FILE NUMBERS OPA 194 [OPA/7279]\n\nTAKE NOTICE that a public meeting will be held to consider amendments to the City of Windsor these\n\n## DEVELOPMENT \\& HERITAGE STANDING COMMITTEE (DHSC) Monday, May 5, 2025 at 4:30 p.m.\n\nPlease Note: Members will have the option of participating in person in Council Chambers or electronically and will be counted towards quorum in accordance with Procedure By-law 98-2011 as amende', " STANDING COMMITTEE (DHSC) Monday, May 5, 2025 at 4:30 p.m.\n\nPlease Note: Members will have the option of participating in person in Council Chambers or electronically and will be counted towards quorum in accordance with Procedure By-law 98-2011 as amended to allow for electronic meetings. The minutes will reflect this accordingly. Any personal information may become part of the public record. To register as a delegate, to appear either in person or by electronic pa

In [37]:
# create embeddings for each chunk
def create_embeddings(chunks):
    # create embeddings for each chunk
    tries = 0
    embeddings = []
    i = 0
    while True:
        tries+=1
        try:
            
            # if len chunks > 96, split into chunks of 96
            while i < len(chunks):
                # create a chunk of 96
                chunk = chunks[i:i+96]
                # create embedding
                response = co.embed(texts=chunk, model='embed-v4.0', input_type='text', embedding_types=['float'])
                embeddings.extend(response.embeddings.float)
                i += 96
            # response = co.embed(texts=chunks, model='embed-v4.0', input_type='text', embedding_types=['float'])
            # embeddings.extend(response.embeddings.float)
            # for chunk in chunks:
            #     # create embedding
            #     embedding = co.embed(texts=[chunk], input type)
            #     # add embedding to list
            #     embeddings.append(embedding)
            return embeddings
        except Exception as e:
            print(f"Error creating embeddings: {e}")
            if tries > 5:
                print("Too many tries, returning None")
                return None
            time.sleep(60)
# save embeddings to a file
def save_embeddings(embeddings, file_path):
    # save embeddings to file
    with open(file_path, 'w') as f:
        json.dump(embeddings, f)


In [28]:
embeddings = create_embeddings(chunks)

In [30]:
print(len(embeddings))

138


In [ ]:
# save_embeddings(embeddings.embeddings.float, os.path.join(chunks_path, project_files[0].replace('.json', '.embeddings.json')))

In [38]:
# iterate through all files in chunks directory and create embeddings for each file
embeddings_path = "./embeddings"
# cycle through all valid json files and create chunks, save chunks and page numbers to file
if not os.path.exists(embeddings_path):
    os.makedirs(embeddings_path)
# extract and save embeddings for each file
valid_chunks = []
for i, file in enumerate(valid_files):
    # check if sub-folder exists
    sub_folder = os.path.join(embeddings_path, os.path.dirname(file))
    if not os.path.exists(sub_folder):
        os.makedirs(sub_folder)
    # if json file already exists, skip it
    if os.path.exists(os.path.join(embeddings_path, file.replace('.json', '.embeddings.json'))):
        print(f"File {i+1}/{len(project_files)}: {file} - already exists, skipping")
        continue
    
    # create chunks
    print(f"Creating embeddings for file {i+1}/{len(project_files)}: {file}")
    with open(os.path.join(chunks_path, file.replace('.json', '.chunks.json')), 'r') as f:
        data = json.load(f)
        chunks = data["chunks"]
        page_numbers = data["page_numbers"]
    
    # create embeddings

    embeddings = create_embeddings(chunks)
    
    # save embeddings and page numbers to file
    with open(os.path.join(embeddings_path, file.replace('.json', '.embeddings.json')), 'w') as f:
        json.dump({"embeddings": embeddings}, f)
    print(f"File {i+1}/{len(project_files)}: {file} - {len(chunks)} chunks created")

File 1/11: 8607-8649-8675-8699-McHugh-Street/Application Form Plan of Subdivision or Condominium.pdf.json - already exists, skipping
File 2/11: 8607-8649-8675-8699-McHugh-Street/Condominium Drawings.pdf.json - already exists, skipping
File 3/11: 8607-8649-8675-8699-McHugh-Street/DPC Exemption Cover Letter (December 2021).pdf.json - already exists, skipping
File 4/11: 0-1095-1185-North-Talbot-Road/SDN-001_21 - 1095 and 1185 North Talbot Survey.pdf.json - already exists, skipping
File 5/11: 0-1095-1185-North-Talbot-Road/SDN-001_21 - Acoustical Report.pdf.json - already exists, skipping
File 6/11: 0-1095-1185-North-Talbot-Road/SDN-001_21 - Bellocorp Development - North Talbot Tree Report.pdf.json - already exists, skipping
File 7/11: 0-1095-1185-North-Talbot-Road/SDN-001_21 - Draft Plan.pdf.json - already exists, skipping
File 8/11: 0-1095-1185-North-Talbot-Road/SDN-001_21 - Endangered Species Act Study.pdf.json - already exists, skipping
File 9/11: 0-1095-1185-North-Talbot-Road/SDN-001_2

KeyboardInterrupt: 